# High Performance Python Notes
> My notes for the book "High Performance Python"
- toc: true 
- badges: false
- comments: true
- categories: [programming]
<!-- - image: images/normal-dist.jpg -->

Making the code faster requires us to first understand where are the bottlenecks in our program. By finding out pieces of code which are taking the longest time we can narrow down our focus to those and save ourselves from spending our time and energy on parts of programme which were already optimized for performance. 

## Profiling to find bottlenecks

Let's say we want to calculate how much time a function `foo` in our program takes to run. We can do this using `time` module and calculating the time taken wherever the function is called

In [ ]:
import time

start = time.time() ## Noting the start time

foo(*args, **kwargs) ## calling the function

end = time.time() ## noting the end time

print(f'The function {foo.__name__} took {end-start} seconds to run')

This approach however requires us to write the code for calculating the time taken by function everywhere the function is called. If the function is called numerous times, this approach can clutter our program.

A better approach would be to use decorator

### Using Decorators

In [2]:
def timer_func(func):
    
    def time_measurer(*args, **kwargs):
        start = time.time()
        
        reult = func(*args, **kwargs)
        
        end = time.time()
        
        print(f'The function {func.__name__} took {end-start} seconds to run')
        
        return result
    
    return time_measurer

Now we only need to "decorate"  the function as follows

In [5]:
@timer_func
def foo(*args, **kwargs):
    ...
    
    

The above code snippet is just a fancy way of saying `foo = timer_func(foo)`.  With this approach, we only need to write the code for calculating the time taken once and then using a decorator we can convert `foo` into a function that prints the time taken and returns the result. Moreover, we can time any function using this decorator

But there's one problem with this approach. Whenever the function `foo` will be called it will print out `The function time_measurer took 10 seconds to run`. This is because `timer_func` returns a function named "time_measurer". We can circumvent this issue by a small fix.

In [6]:
from functools import wraps

def timer_func(func):
    
    @wraps(func)
    def time_measurer(*args, **kwargs):
        start = time.time()
        
        reult = func(*args, **kwargs)
        
        end = time.time()
        
        print(f'The function {func.__name__} took {end-start} seconds to run')
        
        return result
    
    return time_measurer

`wraps` decorator forces the function `time_measurer` to have the same attributes as that of `func`.

### Using magic commands

In Jupyter notebooks we can use magic `%timeit` for timing the function. This will return mean and standard deviation of run time of several calls to the function

In [3]:
import julia_set

%timeit julia_set.calc_pure_python(desired_width = 1000, max_iterations = 300)

17 s ± 1.41 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


`timeit` can also be run from command line as:

In [4]:
! python -m timeit -n 5 -r 2 -s "import julia_set" "julia_set.calc_pure_python(desired_width = 1000, max_iterations = 300)"

5 loops, best of 2: 16.2 sec per loop


> Note: Running `timeit` using magic command return mean of all the runs while while running it from command line displays the time of the best run

### Using cProfile Module

cProfile is the build in profiling tool in the standard library. Using this module gives more detailed information at the cost of greater overhead. It can be used from command line as below

In [6]:
!python -m cProfile -s cumulative julia_set.py

         36221990 function calls in 24.540 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   24.540   24.540 {built-in method builtins.exec}
        1    0.038    0.038   24.540   24.540 julia_set.py:1(<module>)
        1    0.894    0.894   24.503   24.503 julia_set.py:21(calc_pure_python)
        1   15.991   15.991   23.277   23.277 julia_set.py:7(calculate_z_serial_purepython)
 34219980    7.286    0.000    7.286    0.000 {built-in method builtins.abs}
  2002000    0.321    0.000    0.321    0.000 {method 'append' of 'list' objects}
        1    0.010    0.010    0.010    0.010 {built-in method builtins.sum}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




To get more control over the results of cProfile we can write the results into a statistics file as below:

In [7]:
! python -m cProfile -o profile.stats julia_set.py

The above line of code writes the results of cProfile in a file named `profile.stats`. We can analyze this file in a seperate programme using the `pstats` module

In [9]:
import pstats

p = pstats.Stats("profile.stats")

p.sort_stats("cumulative")

The last line of code in the cell above sorted the functions according to the cumulative time taken by them

In [10]:
p.print_stats()

Sat Jul 17 10:32:32 2021    profile.stats

         36221990 function calls in 24.906 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   24.906   24.906 {built-in method builtins.exec}
        1    0.039    0.039   24.906   24.906 julia_set.py:1(<module>)
        1    0.942    0.942   24.867   24.867 julia_set.py:21(calc_pure_python)
        1   16.292   16.292   23.595   23.595 julia_set.py:7(calculate_z_serial_purepython)
 34219980    7.303    0.000    7.303    0.000 {built-in method builtins.abs}
  2002000    0.319    0.000    0.319    0.000 {method 'append' of 'list' objects}
        1    0.011    0.011    0.011    0.011 {built-in method builtins.sum}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Similarly, we can sort according to total time taken as follows:

In [11]:
p.sort_stats("tottime")

In [12]:
p.print_stats()

Sat Jul 17 10:32:32 2021    profile.stats

         36221990 function calls in 24.906 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   16.292   16.292   23.595   23.595 julia_set.py:7(calculate_z_serial_purepython)
 34219980    7.303    0.000    7.303    0.000 {built-in method builtins.abs}
        1    0.942    0.942   24.867   24.867 julia_set.py:21(calc_pure_python)
  2002000    0.319    0.000    0.319    0.000 {method 'append' of 'list' objects}
        1    0.039    0.039   24.906   24.906 julia_set.py:1(<module>)
        1    0.011    0.011    0.011    0.011 {built-in method builtins.sum}
        1    0.000    0.000   24.906   24.906 {built-in method builtins.exec}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




To get a sense of which function was called by which, or what are the functions being profiled we can use `print_callers()` method:

In [13]:
p.print_callers()

   Ordered by: internal time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
julia_set.py:7(calculate_z_serial_purepython)     <-       1   16.292   23.595  julia_set.py:21(calc_pure_python)
{built-in method builtins.abs}                    <- 34219980    7.303    7.303  julia_set.py:7(calculate_z_serial_purepython)
julia_set.py:21(calc_pure_python)                 <-       1    0.942   24.867  julia_set.py:1(<module>)
{method 'append' of 'list' objects}               <- 2002000    0.319    0.319  julia_set.py:21(calc_pure_python)
julia_set.py:1(<module>)                          <-       1    0.039   24.906  {built-in method builtins.exec}
{built-in method builtins.sum}                    <-       1    0.011    0.011  julia_set.py:21(calc_pure_python)
{built-in method builtins.exec}                   <- 
{built-in method builtins.len}                    <-       2    0.000    0.000  juli

To print which function called which other functions i.e flipping the information in previous output cell, we can use `p.print_callees()`

In [14]:
p.print_callees()

   Ordered by: internal time

Function                                          called...
                                                      ncalls  tottime  cumtime
julia_set.py:7(calculate_z_serial_purepython)     -> 34219980    7.303    7.303  {built-in method builtins.abs}
                                                           2    0.000    0.000  {built-in method builtins.len}
{built-in method builtins.abs}                    -> 
julia_set.py:21(calc_pure_python)                 ->       1   16.292   23.595  julia_set.py:7(calculate_z_serial_purepython)
                                                           2    0.000    0.000  {built-in method builtins.len}
                                                           1    0.011    0.011  {built-in method builtins.sum}
                                                     2002000    0.319    0.319  {method 'append' of 'list' objects}
{method 'append' of 'list' objects}               -> 
julia_set.py:1(<module>)            

### Visualising cProfile Output using SnakeViz

We can use `snakeviz` visulaiser to visualize the outputs of cProfile profiler

In [ ]:
!pip install snakeviz

In [ ]:
! snakeviz profile.stats

The above line of produces a graphic as shown below:
![](my_icons/profiler.PNG)

### Using line_profiler for Line-by-Line measurements

Above profiling methods only give information at the function level. That's a good starting point. Once we have figured out the functions which are taking up the most time we can profile them further line by line using the `line_profiler` tool by Robert Kern

In [ ]:
!pip install line_profiler

Before running the relevant line_profiler commands from the command line we have to decorate the functions we are interested in by `@profile` decorator. The `kernprof` script is used to execute the code. The argument `-l` specifies that we want line-by-line profiling instead of function level. The `-v` argument stand for verbose output. Without `-v` you receive an `.lprof` output which can be later analyzed by `line-profiler` module.

In [1]:
! kernprof -l -v julia_set.py

Wrote profile results to julia_set.py.lprof
Timer unit: 1e-06 s

Total time: 93.4638 s
File: julia_set.py
Function: calculate_z_serial_purepython at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           @profile
     8                                           def calculate_z_serial_purepython(maxiter, zs, cs):
     9                                               """Calculate output list using Julia update rule"""
    10         1       5846.4   5846.4      0.0      output = [0] * len(zs)
    11   1000001     656046.7      0.7      0.7      for i in range(len(zs)):
    12   1000000     655067.5      0.7      0.7          n = 0
    13   1000000     750590.3      0.8      0.8          z = zs[i]
    14   1000000     719062.3      0.7      0.8          c = cs[i]
    15  34219980   36962398.8      1.1     39.5          while abs(z) < 2 and n < maxiter:
    16  33219980   27921885.4      0.8     29.9              z = z * z + c

By looking at the `%Time` column we can see that the while loop in function `calc_pure_python` takes 39.5 % of the total time it takes to run the whole programme. More specifically the line `while abs(z) < 2 and n < maxiter` takes up 39.5% of the time. We can't figure out which part of either side of `and` takes the most time yet. Also we can see that an operation as simple as `n += 1` takes up 26% of the total time. This is because of the python's dynamic lookup machinery. Every time we encounter that line in the loop python checks whether the `n` object has an `__add__` function. (If it didn't, it'll walk up any inherited classes to see whether any of them has it.), then integer 1 is passed to the `__add__` function to it's thing. To reduce the cost of this rather simple operation, type specialization is the way to go (which we'll encounter later).

Now getting back to that time consuming while loop; we want to figure out which one of `abs(z) < 2` and `n < maxiter` took greater time. Whichever takes less time and is False more often should be put on the left side of `and` because if first argument of an `and` statement is False it doesn't even need to look at the second argument.

We can breakup the two statements on either send of `and` into seperate lines and then run a line profiler as below:

In [ ]:
statement1 = abs(z) < 2
statement2 = n < maxiter

while statement1 and statement2:
    ...

In [2]:
! kernprof -l -v julia_set.py

Wrote profile results to julia_set.py.lprof
Timer unit: 1e-06 s

Total time: 156.846 s
File: julia_set.py
Function: calculate_z_serial_purepython at line 21

Line #      Hits         Time  Per Hit   % Time  Line Contents
    21                                           @profile
    22                                           def calculate_z_serial_purepython(maxiter, zs, cs):
    23                                               """Calculate output list using Julia update rule"""
    24         1       6964.2   6964.2      0.0      output = [0] * len(zs)
    25   1000001     727030.2      0.7      0.5      for i in range(len(zs)):
    26   1000000     703580.7      0.7      0.4          n = 0
    27   1000000     822355.9      0.8      0.5          z = zs[i]
    28   1000000     795890.2      0.8      0.5          c = cs[i]
    29                                                   while True:
    30  34219980   39121477.8      1.1     24.9              statement1 = abs(z) < 2
    31  34

It looks as if checking `abs(z) < 2` takes more time than checking `n < maxiter`. Now we need to know which of these two statements is `False` more often. `n < maxiter` will be false on 1 of every 301 calls since `maxiter = 300` in the programme. 